In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.dummy import DummyClassifier

# Загружаем данные
data = pd.read_csv("train.csv")


**Обоснование выбора метрики**
Для задачи бинарной классификации на основе данных о выживаемости выбраны следующие метрики:

F1-score: так как классы несбалансированы, она учитывает и точность, и
полноту.

ROC-AUC: так как оценивает разделяющую способность модели, независимо от порога классификации.

In [4]:
# Тренировочная и тестовая выборки

# Удаление ненужных столбцов
data = data.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"])

# Обработка данных

data["Age"] = data["Age"].fillna(data["Age"].median())
data["Embarked"] = data["Embarked"].fillna(data["Embarked"].mode()[0])

data = pd.get_dummies(data, columns=["Sex", "Embarked"], drop_first=True)

X = data.drop("Survived", axis=1)
y = data["Survived"]

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [5]:
# Оценка бейзлайн-модели

# Бейзлайн: DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=42)
dummy_clf.fit(X_train, y_train)

# Предсказания и метрики
y_dummy_pred = dummy_clf.predict(X_test)
print("Бейзлайн:")
print(f"Accuracy: {accuracy_score(y_test, y_dummy_pred):.2f}")
print(f"F1-score: {f1_score(y_test, y_dummy_pred):.2f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_dummy_pred):.2f}")


Бейзлайн:
Accuracy: 0.61
F1-score: 0.00
ROC-AUC: 0.50


In [6]:
# Обучение модели
# Логистическая регрессия
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)

# Предсказания и метрики
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("\nЛогистическая регрессия:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(f"F1-score: {f1_score(y_test, y_pred):.2f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_proba):.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



Логистическая регрессия:
Accuracy: 0.80
F1-score: 0.72
ROC-AUC: 0.84

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       110
           1       0.79      0.67      0.72        69

    accuracy                           0.80       179
   macro avg       0.80      0.78      0.79       179
weighted avg       0.80      0.80      0.80       179

